In [9]:
sc

<SparkContext master=yarn appName=PySparkShell>

In [37]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

ImportError: cannot import name 'DateType' from 'pyspark.sql' (/opt/spark/python/pyspark/sql/__init__.py)

In [11]:
spark = SparkSession.builder.appName('ikhlebushkin_spark_df_task2').master('yarn').getOrCreate()

In [12]:
business = spark.read.json('/data/yelp/business')
checkins = spark.read.json('/data/yelp/checkin')

In [32]:
categories = business.select(
    'business_id',
    f.explode(
        f.split(business.categories, ', ')
    ).alias('category')
)

In [66]:
months = checkins.select('business_id',
                         f.explode(f.split(checkins.date, ', ')).alias('checkin_date'))
months = (months
          .withColumn('mnth', f.date_format(months.checkin_date.cast('date'), 'yyyy-MM'))
          .drop('checkin_date'))

In [90]:
counted = months.groupby('business_id', 'mnth').count()

In [103]:
joined_df = (categories
             .join(counted, 'business_id', 'inner')
             .drop('business_id')
             .groupBy('category', 'mnth')
             .agg(f.sum('count').alias('checkins'))
             .orderBy('mnth', 'category')
             .select('mnth', 'category', 'checkins'))

In [104]:
joined_df.head(50)

[Row(mnth='2009-12', category='American (Traditional)', checkins=1),
 Row(mnth='2009-12', category='Breakfast & Brunch', checkins=1),
 Row(mnth='2009-12', category='Diners', checkins=1),
 Row(mnth='2009-12', category='Indian', checkins=1),
 Row(mnth='2009-12', category='Restaurants', checkins=2),
 Row(mnth='2010-01', category='Accessories', checkins=6),
 Row(mnth='2010-01', category='Active Life', checkins=44),
 Row(mnth='2010-01', category='Adult Education', checkins=2),
 Row(mnth='2010-01', category='Adult Entertainment', checkins=2),
 Row(mnth='2010-01', category='Airlines', checkins=4),
 Row(mnth='2010-01', category='Airports', checkins=27),
 Row(mnth='2010-01', category='Allergists', checkins=1),
 Row(mnth='2010-01', category='Amateur Sports Teams', checkins=3),
 Row(mnth='2010-01', category='American (New)', checkins=245),
 Row(mnth='2010-01', category='American (Traditional)', checkins=232),
 Row(mnth='2010-01', category='Amusement Parks', checkins=7),
 Row(mnth='2010-01', categ

In [ ]:
negative_tops.write.mode('overwrite').csv('spark_df_task1', sep='\t')

In [ ]:
spark.stop()